In [138]:
import pandas as pd
import numpy as np
import pickle
from sys import getsizeof

data_dir = 'data/final_data/'

In [2]:
data_r = pd.read_csv(data_dir + 'data_binary.csv')

data_cols = data_r.columns
colsr1 = data_cols[0:20]
# colsr2 = data_cols[3:20]
colsr3 = data_cols[26:29]
data_cols_r1 = colsr1.append(colsr3)

keep = ['FIPST', 'LSTATE']

data_cols_r1 = [i for i in data_cols_r1 if i not in keep]

data_cols_r1

/home/chrisleetran94/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (5,6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


['SURVYEAR',
 'NCESSCH',
 'LEAID',
 'SCHNO',
 'STID',
 'SEASCH',
 'LEANM',
 'SCHNAM',
 'PHONE',
 'MSTREE',
 'MCITY',
 'MSTATE',
 'MZIP',
 'MZIP4',
 'LSTREE',
 'LCITY',
 'LZIP',
 'LZIP4',
 'CONUM',
 'CONAME',
 'CDCODE']

In [3]:
data_n = data_r.drop(data_cols_r1, axis=1)
print(data_n.shape)
data_n = data_n.rename(columns={'FIPST': 'state', 'LSTATE': 'state_name'})
data_n['index'] = data_n.index

(36295, 421)


In [4]:
# data_n[8520:8540]

In [5]:
def distance(lat1, lon1, lat2, lon2):
    R = 3961

    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    distance = R * c
    
    return distance

In [128]:
def create_network(data_n, percent_edges=.2):
    
    dist_eps=.01
    
    norm_eps1=.01
    norm_eps2=.1
    
    adjacency = list()
    edge_weights = list()

    for i in range(data_n.shape[0]):
        
        idx = (data_n['state'] == data_n['state'][i]) & (data_n['index'] != i)
        
        edge_idx = data_n['index'][idx].as_matrix()

        dist = 1/(distance(data_n['LATCOD'][idx], data_n['LONCOD'][idx], data_n['LATCOD'][i], data_n['LONCOD'][i]) + dist_eps)
        dist = (dist - dist.min() + norm_eps1)/(dist.max() - dist.min() + norm_eps2)
        dist = dist.as_matrix()

        sort_idx = np.argsort(dist)[::-1]

        sort_idx = sort_idx[:np.int(np.ceil(percent_edges*sort_idx.shape[0]))]

        adjacency.append(edge_idx[sort_idx])
        edge_weights.append(dist[sort_idx])
        
    return adjacency, edge_weights

In [129]:
# dist_eps=.01
    
# norm_eps1=.01
# norm_eps2=.1

# idx = (data_n['state'] == data_n['state'][i]) & (data_n['index'] != i)

# dist = 1/(distance(data_n['LATCOD'][idx], data_n['LONCOD'][idx], data_n['LATCOD'][i], data_n['LONCOD'][i]) + dist_eps)
# dist = (dist - dist.min() + norm_eps1)/(dist.max() - dist.min() + norm_eps2)
# dist = dist.as_matrix()

# edge_idx = data_n['index'][idx].as_matrix()

# sort_idx = np.argsort(dist)[::-1]

# sort_idx = sort_idx[:np.int(np.ceil(percent_edges*sort_idx.shape[0]))]

# adjacency.append(edge_idx[sort_idx])
# edge_weights.append(dist[sort_idx])

# print(edge_idx[sort_idx])
# print(sort_idx)

In [130]:
%%time

with open(data_dir + 'network.pkl', 'wb') as f:
    adjacency, edge_weights = create_network(data_n)
    adjacency = np.array(adjacency)
    edge_weights = np.array(edge_weights)
    pickle.dump([adjacency, edge_weights], f, protocol=-1)

CPU times: user 2min 10s, sys: 484 ms, total: 2min 11s
Wall time: 2min 11s


In [159]:
data_nf = data_n.drop(['index'], axis=1)

data_nf.to_csv(data_dir + 'data_n', index=False, encoding='utf-8')

In [170]:
# dist_eps=.01
    
# norm_eps1=.01
# norm_eps2=.1

# adjacency = list()
# edge_weights = list()

# i = 0
# percent_edges=.2

# idx = (data_n['state'] == data_n['state'][i]) & (data_n['index'] != i)

# edge_idx = data_n['index'][idx].as_matrix()

# dist = 1/(distance(data_n['LATCOD'][idx], data_n['LONCOD'][idx], data_n['LATCOD'][i], data_n['LONCOD'][i]) + dist_eps)
# dist = (dist - dist.min() + norm_eps1)/(dist.max() - dist.min() + norm_eps2)
# dist = dist.as_matrix()

# sort_idx = np.argsort(dist)[::-1]

# sort_idx = sort_idx[:np.int(np.ceil(percent_edges*sort_idx.shape[0]))]

# adjacency.append(edge_idx[sort_idx])
# edge_weights.append(dist[sort_idx])

# edge_weights[0].shape